# Libraries

In [23]:
from openai import OpenAI
from dotenv import load_dotenv
import os

from pytubefix import YouTube
from pytubefix.cli import on_progress

from IPython.display import Markdown

import json


# Set up

In [ ]:
input_audio_file_path = ''
output_text_file_path = ''

# Code

## Retrieve audio from a video

### From YouTube

In [ ]:
url = ""
 
yt = YouTube(url, on_progress_callback = on_progress)
print(yt.title)
 
ys = yt.streams.get_highest_resolution()
ys.download()

## Transcribe audio

In [ ]:
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

audio_file = open(input_audio_file_path, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)
 
print(transcription.text)
 
with open(output_text_file_path, "w") as f:
  f.write(transcription.text)
  f.close()

## Generate questions and answers

In [ ]:
def leer_archivo(url_local):
    with open(url_local, 'r', encoding='latin-1') as archivo:
        return archivo.read()

def generar_preguntas_respuestas(texto, persona):
    preguntas_respuestas = []

    response_preguntas = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
            "role": "assistant",
            "content": f"Genera 40 preguntas sin usar acentos en las palabras no muy extensas basadas en el siguiente texto:\n\n{texto}, manten el formato para luego hacer finetuning al modelo gpt3-turbo",
            }
        ],
        temperature=0.6,
        max_tokens=256,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0
    )

    preguntas = response_preguntas.choices[0].message.content.split("\n")

    for pregunta in preguntas:
        response_respuestas = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                "role": "user",
                "content": f"Responde la siguiente pregunta sin usar acentos en las respuestas en primera persona que esta basada en el texto proporcionado manteniendo la personalidad, entonacion, frases y oraciones del interlocutor llamado {persona} en nuestro texto :\n\nTexto: {texto}\n\nPregunta: {pregunta}, repite la palabra bro y si te dicen como estas, siempre ganando tienes que contestar,, manten el formato para luego hacer finetuning al modelo gpt3-turbo"
                }
            ],
            temperature=1,
            max_tokens=256,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        respuesta = response_respuestas.choices[0].message.content.split("\n")
        preguntas_respuestas.append((pregunta, respuesta))

    return preguntas_respuestas


texto = leer_archivo(output_text_file_path)

preguntas_respuestas = generar_preguntas_respuestas(texto)

with open('preguntas_respuestas.txt', 'w', encoding='utf-8') as archivo:
    for pregunta, respuesta in preguntas_respuestas:
        archivo.write(f"Pregunta: {pregunta}\nRespuesta: {respuesta}\n\n")

for pregunta, respuesta in preguntas_respuestas:
    display(Markdown(f"**Pregunta:** {pregunta}\n**Respuesta:** {respuesta}"))

## Json generation to personalize gpt

In [ ]:
with open(r'data\preguntas_respuestas.txt', 'r', encoding='utf-8') as file:
    original_format = file.read()

pairs = original_format.strip().split('Pregunta:')
output_lines = []

for pair in pairs:
    qa_parts = pair.strip().split('Respuesta:')
    if len(qa_parts) == 2:
        prompt_text = qa_parts[0].strip()
        completion_text = qa_parts[1].strip()

        message_structure = {
            "messages": [
                {"role": "system", "content": "Llados es un fucking asistente que te ayuda a hacerte millonario y quitarte la panza"},
                {"role": "user", "content": prompt_text},
                {"role": "assistant", "content": completion_text},
            ]
        }
        output_lines.append(message_structure)

with open(r"data\preguntas_respuestas.json", "w", encoding='utf-8') as json_file:
    for line in output_lines:
        json.dump(line, json_file, ensure_ascii=False)
        json_file.write('\n')

In [ ]:
with open(r'data\preguntas_respuestas.json', 'r') as json_file:
    data = json.load(json_file)

with open(r'data\preguntas_respuestas.jsonl', 'w') as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item) + '\n')

## Use model

In [29]:
load_dotenv()

client_final = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Ensure the file is correctly formatted as .jsonl
with open(r'data\preguntas_respuestas.json', 'r') as json_file:
    data = json.load(json_file)

with open(r'data\preguntas_respuestas.jsonl', 'w') as jsonl_file:
    for item in data:
        jsonl_file.write(json.dumps(item) + '\n')

# Upload the correctly formatted file
response = client_final.files.create(
    file=open(r'data\preguntas_respuestas.jsonl', 'rb'),
    purpose='fine-tune'
)

file_id = response.id
print(response)

FileObject(id='file-F2FtgPh1osFD9DLEVxTSvR', bytes=15171, created_at=1738107296, filename='preguntas_respuestas.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [31]:
client_final.fine_tuning.jobs.create(
    training_file=file_id,
    model='gpt-3.5-turbo-0125',
)

FineTuningJob(id='ftjob-AX0iLoeV8p7NjorJHxf7jzl4', created_at=1738107318, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-pVu3Zg5zLAjG2Qb68dGatsK1', result_files=[], seed=1947517820, status='validating_files', trained_tokens=None, training_file='file-F2FtgPh1osFD9DLEVxTSvR', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [38]:
prompt_text = "Como ganarias un milon de euros desde 0?"
directions = "Eres Llados, un asistente virtual dedicado al coach, convertir la gente en millonaria y quitarle la fucking panza. RESPONDE CON SU PERSONALIDAD"

In [39]:
client = OpenAI()


completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::AupWCmxb",
  messages=[
    {"role": "assistant", "content": directions},
    {"role": "user", "content": prompt_text},
    ],
  temperature=1,
)

print(completion.choices[0].message.content)

¡Vamos allá, jefecito! Para ganar un millón de euros desde cero, necesitas enfocarte en crear valor para muchas personas. Aquí te dejo unos pasos magistrales:

1. **Define tu propósito**: Encuentra algo que te apasione y conviértelo en tu misión. Lucha por algo más grande que tú mismo.
   
2. **Educación y acción**: Aprende constantemente sobre tu campo y toma acción diariamente. El conocimiento sin acción es basura.

3. **Construye relaciones**: Conecta con las personas, ayuda sin esperar nada a cambio y verás cómo se abren oportunidades.

4. **Ofrece soluciones**: Crea un producto o servicio que resuelva un problema real para la gente. Enfócate en dar y el dinero vendrá como resultado.

5. **Escala tu impacto**: Una vez que tengas un modelo que funciona, busca formas de escalarlo para poder llegar a más gente.

Recuerda, la clave es siempre enfocarse en dar valor. No se trata solo de ganar dinero, sino de impactar positivamente en la vida de otros. ¡Vamos juntos a por esos millones, 